### Segmenting and Clustering Neighborhoods in Toronto

In [29]:
#import libraries

import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import json
import requests

In [6]:
source = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [8]:
text_results = requests.get(source).text

In [10]:
#transforming the text into html
html_parsered_result = BeautifulSoup(text_results, 'html.parser')

In [11]:
neighbourhood_info_table = html_parsered_result.find('table', class_ = 'wikitable')
neighbourhood_rows = neighbourhood_info_table.find_all('tr')

# extracting the information ('Postcode', 'Borough', 'Neighbourhood') from the table
neighbourhood_info = []
for row in neighbourhood_rows:
    info = row.text.split('\n')[1:-1]
    neighbourhood_info.append(info)
    
neighbourhood_info[0:10]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned']]

In [22]:
neighbourhood_info[0][-1] = 'Neighborhood' 
dataframe = pd.DataFrame(neighbourhood_info[1:], columns=neighbourhood_info[0])

dataframe.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only processing the cells that have an assigned borough. Ignoreing cells with a borough that is Not assigned.

In [24]:
not_assigned_boroughs = dataframe.index[dataframe['Borough'] == 'Not assigned']

dataframe.drop(dataframe.index[not_assigned_boroughs], inplace=True)
dataframe.reset_index(drop=True, inplace=True)

dataframe.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


The dataframe grouped by Postcode

In [27]:
group = dataframe.groupby('Postcode')
grouped_neighbour = group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_dataframe = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighbour)))
grouped_dataframe.columns = ['Postcode', 'Borough', 'Neighborhood']

grouped_dataframe

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [28]:
grouped_dataframe.shape

(103, 3)